In [ ]:
import sys
import time
from datetime import date
from dateutil.relativedelta import relativedelta
from pathlib import Path

import requests
import pandas as pd
import mlflow
from domino.data_sources import DataSourceClient

API_BASE   = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query"
PAGE_SIZE  = 10000   # plenty for a single year
BATCH_SIZE = 5000     # rows per upsert batch

mlflow.set_experiment("Populate DB (treasury_auction_results)")
ds = DataSourceClient().get_datasource("market_data")

def fetch_all_auctions_last_y(years_to_backfill) -> list[dict]:
    rows = []
    start_year = (date.today() - relativedelta(years=years_to_backfill)).year
    end_year   = date.today().year

    for yr in range(start_year, end_year + 1):
        print('year', yr)
        start = date(yr, 1, 1).isoformat()
        end   = date(yr, 12, 31).isoformat()
        params = [
            ("filter", f"record_date:gte:{start}"),
            ("filter", f"record_date:lte:{end}"),
            ("page[size]", str(PAGE_SIZE)),
            ("sort", "record_date"),
        ]
        resp = requests.get(API_BASE, params=params)
        resp.raise_for_status()
        data = resp.json().get("data", [])
        print(f"  {yr}: fetched {len(data)} rows")
        rows.extend(data)
    return rows

def quote_value(val):
    if val is None or val == "null":
        return "NULL"
    try:
        float(val)
        return str(val)
    except:
        return "'" + str(val).replace("'", "''") + "'"

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

def dedupe(records: list[dict]) -> list[dict]:
    seen = set()
    deduped = []
    for r in records:
        key = (r.get("record_date"), r.get("cusip"))
        if key in seen:
            continue
        seen.add(key)
        deduped.append(r)
    return deduped

def upsert_batch(records: list[dict], batch_size: int = BATCH_SIZE):
    if not records:
        return
    cols = list(records[0].keys())
    col_list = ", ".join(cols)
    total = len(records)
    print(f"Upserting {total} rows in batches of {batch_size}...")
    for idx, chunk in enumerate(chunks(records, batch_size), start=1):
        print(f"  Batch {idx}: {len(chunk)} rows")
        values_sql = []
        for r in chunk:
            vals = [quote_value(r.get(c)) for c in cols]
            values_sql.append("(" + ", ".join(vals) + ")")
        values_str = ",\n".join(values_sql)
        set_list = ", ".join([f"{c} = EXCLUDED.{c}" for c in cols])
        sql = f"""
        INSERT INTO treasury_auction_results ({col_list})
        VALUES
        {values_str}
        ON CONFLICT (record_date, cusip)
        DO UPDATE SET
          {set_list};
        """
        ds.query(sql)

def main(years_to_backfill):
    with mlflow.start_run(run_name="Load 30yr Auction Results"):
        t0 = time.time()

        data = fetch_all_auctions_last_y(years_to_backfill)
        mlflow.log_metric("rows_fetched", len(data))
        data = dedupe(data)
        upsert_batch(data)

        duration = time.time() - t0
        mlflow.log_metric("duration_seconds", duration)
        print(f"✅ Loaded {len(data)} rows in {duration:.1f}s")

        df = pd.DataFrame(data)
        out = Path("artifacts") / "results" / "treasury_auction_results.csv"
        out.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(out, index=False)
        mlflow.log_artifact(str(out), artifact_path="treasury_auction_results")

if __name__ == "__main__":
    main(years_to_backfill = 30)


year 1995
  1995: fetched 8110 rows
year 1996
  1996: fetched 7945 rows
year 1997
  1997: fetched 7775 rows
year 1998
  1998: fetched 7611 rows
year 1999
  1999: fetched 7452 rows
year 2000
  2000: fetched 7297 rows
year 2001
  2001: fetched 7153 rows
year 2002
  2002: fetched 6990 rows
year 2003
  2003: fetched 6803 rows
year 2004
  2004: fetched 6602 rows
year 2005
  2005: fetched 6382 rows
year 2006
  2006: fetched 6163 rows
year 2007
  2007: fetched 5945 rows
year 2008
  2008: fetched 5725 rows
year 2009
  2009: fetched 5464 rows
year 2010
  2010: fetched 5177 rows
year 2011
  2011: fetched 4876 rows
year 2012
  2012: fetched 4610 rows
year 2013
  2013: fetched 4343 rows
year 2014
  2014: fetched 4079 rows
year 2015
  2015: fetched 3809 rows
year 2016
  2016: fetched 3534 rows
year 2017
  2017: fetched 3271 rows
year 2018
  2018: fetched 2994 rows
year 2019
  2019: fetched 2712 rows
year 2020
  2020: fetched 2385 rows
year 2021
  2021: fetched 1883 rows
year 2022
  2022: fetched 14